In [ ]:
# Import libraries and helper functions
import torch
import torchvision.transforms as transforms
import torchvision.models as models
import torch.backends.cudnn as cudnn
import torch.nn as nn
import torch.nn.functional as nnf

import matplotlib.pyplot as plt
import random
import pickle
import pandas as pd

from dataprocessing import *
import sys
sys.path.append('..')
from utils import *
from helper_functions import *


import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import time
import scipy.spatial.distance
from scipy.spatial.distance import hamming
from scipy.spatial import distance

# set hyper parameters here
device = 'cuda' if torch.cuda.is_available() else 'cpu'

class Args:
    batchsize = 100
    model = "mobilenet_v2"
    lr = 0.001
    epochs = 200
    dataset = "tinyimagenet"
    max_noise_rate = 0.4
args = Args()

In [ ]:
# Generate the dataset containing hard and noisy samples

normalize = transforms.Normalize(
      mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    
data_transforms = {
    'train': transforms.Compose([
        #transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
    ]),
    'val': transforms.Compose([
        transforms.ToTensor(),
    ]),
}

data_dir = './tiny-imagenet-200'

# Generating the data augmentations here
#torch.random.manual_seed(1)
#random.seed(1)
transform_aug = [transforms.RandomHorizontalFlip(p=1),
                 transforms.RandomVerticalFlip(p=1),
                transforms.Grayscale(num_output_channels=3),
                transforms.RandomPerspective(distortion_scale=0.5,p=1),
                transforms.RandomPerspective(distortion_scale=0.25,p=1),
                transforms.RandomPerspective(distortion_scale=0.75,p=1),
                transforms.RandomRotation(10),
                transforms.RandomRotation(20),
                transforms.RandomRotation(30),
                transforms.RandomRotation(40),
                transforms.RandomRotation(50),
                transforms.GaussianBlur(3),
                transforms.GaussianBlur(5),
                transforms.ColorJitter(brightness=.5, hue=.3),
                transforms.ColorJitter(brightness=.3, hue=.5),
                transforms.RandomAffine(degrees=(30, 70), translate=(0.1, 0.3), scale=(0.5, 0.75))
                ]

trainset = TinyImageNetAugNoisyDataset(data_dir,transform =data_transforms['train'], normalize=normalize, mode='train'
                                  , transform_aug = transform_aug, max_noise_rate=args.max_noise_rate)

trainloader = torch.utils.data.DataLoader(trainset, batch_size=args.batchsize,
                                             shuffle=True, num_workers=0)

testset = TinyImageNetDataset(data_dir,transform =data_transforms['val'], normalize=normalize, mode='val')
testloader = torch.utils.data.DataLoader(testset, batch_size=args.batchsize,
                                             shuffle=True, num_workers=0)

In [ ]:
# Model and optimizer definitions
pretrained = False
net = models.__dict__[args.model](
        aux_logits=False,pretrained=pretrained, num_classes=num_classes) if args.model == 'inception_v3' else models.__dict__[args.model](pretrained=pretrained, num_classes=num_classes)
net = net.to(device)
if device == 'cuda':
    #net = torch.nn.DataParallel(net)
    cudnn.benchmark = True

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(net.parameters(), lr=args.lr,
                      momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=args.epochs)

In [ ]:
# Train the model on the hard and noisy dataset
file_name = './results/values/hardness_via_diversification_{}_{}_epochs_{}_lr_{}_noise_{}_deployment.pkl'.\
    format(args.dataset, args.model, args.epochs, args.lr, args.max_noise_rate)
model_file_name = './checkpoint/hardness_via_diversification_{}_{}_epochs_{}_lr_{}_noise_{}_deployment.pth'.\
    format(args.dataset, args.model, args.epochs, args.lr, args.max_noise_rate)

with open(file_name, 'wb') as f:
    pickler = pickle.Pickler(f)
    for epoch in range(args.epochs):
        tl, ta, ta5, df_perepoch = train(epoch, net, trainloader, device, criterion, optimizer, model_name=args.model, num_classes=200)
        tel, tea, tea5 = test(epoch, net, testloader, device, criterion)
        scheduler.step()
        pickler.dump([epoch, tl, ta, ta5, df_perepoch, tel, tea, tea5])

    state = {
        'net': net.state_dict(),
        'opt': optimizer.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
torch.save(state, model_file_name)